In [1]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)

base_path: /home/toc3/temp-sr-press


In [2]:
import express.config as config
from express.databases import SQLiteDatabase
from express.datasets import PressingDataset
from express.utils import add_names

In [3]:
TRAIN_DB_PATH = os.path.join(base_path, "stores/train_database.sqlite")
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")

train_db = SQLiteDatabase(TRAIN_DB_PATH)
test_db = SQLiteDatabase(TEST_DB_PATH)

print("train_db:", train_db)
print("test_db:", test_db)

train_db: <express.databases.sqlite.SQLiteDatabase object at 0x7efc9af492e0>
test_db: <express.databases.sqlite.SQLiteDatabase object at 0x7efd6c17ce20>


In [4]:
game_id = 3895302

df_actions = add_names(train_db.actions(game_id))
df_actions

original_event_id  period_id  \
game_id action_id                                                    
3895302 0          221b0c8d-6386-4ae8-bb4a-a1dc98742312          1   
        1          77809242-1460-4395-8779-94a0cfc275b1          1   
        2          ff56e821-21e9-4cef-ba2a-7eb5eb3769c6          1   
        3          cf9088bc-7e59-4d57-8ac5-31658da858bb          1   
        4          4464cb75-f45f-4508-8444-2560d1625d06          1   
...                                                 ...        ...   
        2855       f908d853-a3e9-4ad7-99b4-6aef1f3e3286          2   
        2856       4e61f75d-564b-4478-a2f0-4b7e20e8600e          2   
        2857       91466764-aec7-4fac-a024-dd3deee3c737          2   
        2858       e75b1a76-8aca-461f-846f-70b8b9e35fa9          2   
        2859       dcc0c591-7ebd-4e42-8206-04bfefa94f33          2   

                   time_seconds  team_id  player_id  start_x  start_y  \
game_id action_id                                                       
3895302 0                 3.417      176      34870  52.0625   33.660   
        1                 3.870      176      12299  53.8125   34.340   
        2                 4.732      176      12299  53.8125   34.085   
        3                 6.728      176      31100  74.7250   36.295   
        4                 7.622      176      31100  74.7250   36.720   
...                         ...      ...        ...      ...      ...   
        2855           2660.647      176      12299  55.1250   44.795   
        2856           2661.113      904      10336  57.4000   45.220   
        2857           2662.580      904      40724  77.7875   21.335   
        2858           2662.844      176      24179  84.0875   19.890   
        2859           2665.079      904      40724  92.6625   26.945   

                      end_x    end_y  bodypart_id  ...  play_pattern_name  \
game_id action_id                                  ...                      
3895302 0           53.8125  34.3400            5  ...      From Kick Off   
        1           53.8125  34.0850            0  ...      From Kick Off   
        2           74.7250  36.2950            5  ...      From Kick Off   
        3           74.7250  36.7200            0  ...      From Kick Off   
        4           76.3875  53.8050            5  ...      From Kick Off   
...                     ...      ...          ...  ...                ...   
        2855        55.1250  44.7950            0  ...       From Counter   
        2856        77.7875  21.3350            4  ...       From Counter   
        2857        92.6625  26.9450            0  ...       From Counter   
        2858        84.0875  19.8900            0  ...       From Counter   
        2859       104.5625  35.9975            5  ...       From Counter   

                   under_pressure  duration  \
game_id action_id                             
3895302 0                       0  0.453249   
        1                       0  0.862528   
        2                       0  1.995802   
        3                       0  0.893833   
        4                       0  1.414672   
...                           ...       ...   
        2855                    0  0.206468   
        2856                    0  1.467394   
        2857                    1  2.498700   
        2858                    0  2.214317   
        2859                    0  0.818232   

                                                               extra  \
game_id action_id                                                      
3895302 0          {'pass': {'recipient': {'id': 12299, 'name': '...   
        1                  {'carry': {'end_location': [59.0, 40.6]}}   
        2          {'pass': {'recipient': {'id': 31100, 'name': '...   
        3                  {'carry': {'end_location': [35.1, 43.7]}}   
        4          {'pass': {'recipient': {'id': 51769, 'name': '...   
...                                                        

In [5]:
from express.features import state, pressure
from express import labels as ls

state_features = [f.__name__ for f in state.all_features]
pressure_state_features = [f.__name__ for f in pressure.pressure_state_features]
pressure_with_context_features = [f.__name__ for f in pressure.pressure_with_context_features]
all_labels = [f.__name__ for f in ls.all_labels]

print("state_features:", state_features)
print("pressure_state_features:", pressure_state_features)
print("pressure_with_context_features:", pressure_with_context_features)

print("Labels:", all_labels)

state_features: ['actiontype', 'result', 'bodypart', 'time', 'startlocation', 'endlocation', 'startpolar', 'endpolar', 'movement', 'team', 'time_delta', 'space_delta', 'relative_startlocation', 'relative_endlocation', 'angle', 'speed', 'player_possession_time', 'goalscore', '_opponents_in_3m_radius', 'dist_opponent', 'closest_11_players']
pressure_state_features: ['_opponents_in_3m_radius', 'dist_opponent', 'closest_11_players']
pressure_with_context_features: ['space_delta', 'relative_defender_angle', 'time_delta']
Labels: ['concede_shots', 'possession_change_by_seconds', 'possession_change_by_1_actions', 'possession_change_by_2_actions', 'possession_change_by_3_actions', 'possession_change_by_4_actions', 'possession_change_by_5_actions', 'possession_change_by_6_actions', 'possession_change_by_7_actions', 'possession_change_by_8_actions', 'possession_change_by_9_actions', 'possession_change_by_10_actions', 'possession_change_by_1_actions_and_5m_distance', 'possession_change_by_2_actio

In [6]:
train_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "datasets", "train"),
    state_xfns=state.all_features,
    pressure_xfns={"pressure_state_xfns": pressure.pressure_state_features,
                    "pressure_with_context_xfns": pressure.pressure_with_context_features},
    yfns=[f.__name__ for f in ls.all_labels],
    load_cached =False,
    nb_prev_actions = 3,
)

test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "datasets", "test"),
    state_xfns=state.all_features,
    pressure_xfns={"pressure_state_xfns": pressure.pressure_state_features,
                    "pressure_with_context_xfns": pressure.pressure_with_context_features},
    yfns=[f.__name__ for f in ls.all_labels],
    load_cached =False,
    nb_prev_actions = 3,
)


In [7]:
train_dataset.create(train_db)
test_dataset.create(test_db)

100%|██████████| 64/64 [14:59<00:00, 14.06s/it]


In [14]:
train_dataset.features

actiontype_a0 actiontype_a1     actiontype_a2 result_a0  \
game_id action_id                                                           
3788741 2                  pass       dribble              pass   success   
        18                 pass       dribble              pass      fail   
        26                 pass      throw_in            tackle   success   
        30              dribble     bad_touch           dribble   success   
        33              dribble          pass           dribble   success   
...                         ...           ...               ...       ...   
3943043 2157               pass          pass           dribble   success   
        2167            dribble          pass              pass   success   
        2182           throw_in      goalkick              shot   success   
        2185           throw_in          pass          throw_in   success   
        2206            dribble     clearance  freekick_crossed   success   

                  result_a1 result_a2 bodypart_a0 bodypart_a1 bodypart_a2  \
game_id action_id                                                           
3788741 2           success   success        foot        foot        foot   
        18          success   success        foot        foot        foot   
        26          success      fail        foot       other        foot   
        30             fail   success        foot        foot        foot   
        33          success   success        foot        foot        foot   
...                     ...       ...         ...         ...         ...   
3943043 2157        success   success        foot        foot        foot   
        2167           fail      fail        foot        foot        foot   
        2182           fail      fail       other        foot        head   
        2185           fail   success       other        foot       other   
        2206        success      fail        foot        head        foot   

                   period_id_a0  ...  opponent_10_y_p0  \
game_id action_id                ...                     
3788741 2                     1  ...               NaN   
        18                    1  ...               NaN   
        26                    1  ...               NaN   
        30                    1  ...               NaN   
        33                    1  ...               NaN   
...                         ...  ...               ...   
3943043 2157                  2  ...               NaN   
        2167                  2  ...               NaN   
        2182                  2  ...               NaN   
        2185                  2  ...               NaN   
        2206                  2  ...         29.847266   

                   opponent_10_distance_p0  opponent_11_x_p0  \
game_id action_id                                              
3788741 2                              NaN               NaN   
        18                             NaN               NaN   
        26                             NaN               NaN   
        30                             NaN               NaN   
        33                             NaN               NaN   
...                                    ...               ...   
3943043 2157                           NaN               NaN   
        2167                           NaN               NaN   
        2182                           NaN               NaN   
        2185                           NaN               NaN   
        2206                     34.477067               NaN   

                   opponent_11_y_p0  opponent_11_distance_p0    dx_p0   dy_p0  \
game_id action_id                                                               
3788741 2                       NaN                      NaN  19.6875 -57.290   
        18                      NaN                      NaN   2.3625  -3.910   
        26                      NaN                      NaN  53.3750  43.690   
        30           

In [15]:
train_dataset.labels

concede_shots  possession_change_by_seconds  \
game_id action_id                                                
3788741 2                  False                         False   
        18                 False                         False   
        26                 False                          True   
        30                 False                         False   
        33                 False                         False   
...                          ...                           ...   
3943043 2157                True                         False   
        2167                True                         False   
        2182               False                         False   
        2185               False                         False   
        2206               False                         False   

                   possession_change_by_1_actions  \
game_id action_id                                   
3788741 2                                   False   
        18                                  False   
        26                                  False   
        30                                  False   
        33                                  False   
...                                           ...   
3943043 2157                                False   
        2167                                False   
        2182                                False   
        2185                                False   
        2206                                False   

                   possession_change_by_2_actions  \
game_id action_id                                   
3788741 2                                   False   
        18                                  False   
        26                                  False   
        30                                  False   
        33                                  False   
...                                           ...   
3943043 2157                                False   
        2167                                False   
        2182                                False   
        2185                                False   
        2206                                False   

                   possession_change_by_3_actions  \
game_id action_id                                   
3788741 2                                   False   
        18                                  False   
        26                                  False   
        30                                  False   
        33                                  False   
...                                           ...   
3943043 2157                                False   
        2167                                False   
        2182                                 True   
        2185                                 True   
        2206                                False   

                   possession_change_by_4_actions  \
game_id action_id                                   
3788741 2                                    True   
        18                                  False   
        26                                   True   
        30                                  False   
        33                                  False   
...                                           ...   
3943043 2157                                False   
        2167                                False   
        2182                                 True   
        2185                                 True   
        2206                                False   

                   possession_change_by_5_actions  \
game_id action_id                                   
3788741 2                                    True   
        18                                  False   
        26                                   True   
        30                                  False   
        33                                  False   
...                          

In [16]:
test_dataset.labels

concede_shots  possession_change_by_seconds  \
game_id action_id                                                
3857254 3                  False                         False   
        13                 False                         False   
        15                 False                         False   
        19                 False                         False   
        31                 False                         False   
...                          ...                           ...   
3869685 2913                True                         False   
        2919                True                         False   
        2930               False                         False   
        2937               False                         False   
        2940               False                         False   

                   possession_change_by_1_actions  \
game_id action_id                                   
3857254 3                                   False   
        13                                  False   
        15                                  False   
        19                                  False   
        31                                  False   
...                                           ...   
3869685 2913                                False   
        2919                                False   
        2930                                False   
        2937                                False   
        2940                                False   

                   possession_change_by_2_actions  \
game_id action_id                                   
3857254 3                                   False   
        13                                  False   
        15                                  False   
        19                                  False   
        31                                  False   
...                                           ...   
3869685 2913                                False   
        2919                                False   
        2930                                False   
        2937                                False   
        2940                                False   

                   possession_change_by_3_actions  \
game_id action_id                                   
3857254 3                                   False   
        13                                  False   
        15                                  False   
        19                                  False   
        31                                  False   
...                                           ...   
3869685 2913                                False   
        2919                                False   
        2930                                False   
        2937                                False   
        2940                                False   

                   possession_change_by_4_actions  \
game_id action_id                                   
3857254 3                                   False   
        13                                  False   
        15                                  False   
        19                                  False   
        31                                  False   
...                                           ...   
3869685 2913                                False   
        2919                                False   
        2930                                False   
        2937                                False   
        2940                                False   

                   possession_change_by_5_actions  \
game_id action_id                                   
3857254 3                                   False   
        13                                  False   
        15                                  False   
        19                                  False   
        31                                  False   
...                          

In [11]:
test_dataset.labels

concede_shots  possession_change_by_seconds  \
game_id action_id                                                
3857254 3                  False                         False   
        13                 False                         False   
        15                 False                         False   
        19                 False                         False   
        31                 False                         False   
...                          ...                           ...   
3869685 2913                True                         False   
        2919                True                         False   
        2930               False                         False   
        2937               False                         False   
        2940               False                         False   

                   possession_change_by_1_actions  \
game_id action_id                                   
3857254 3                                   False   
        13                                  False   
        15                                  False   
        19                                  False   
        31                                  False   
...                                           ...   
3869685 2913                                False   
        2919                                False   
        2930                                False   
        2937                                False   
        2940                                False   

                   possession_change_by_2_actions  \
game_id action_id                                   
3857254 3                                   False   
        13                                  False   
        15                                  False   
        19                                  False   
        31                                  False   
...                                           ...   
3869685 2913                                False   
        2919                                False   
        2930                                False   
        2937                                False   
        2940                                False   

                   possession_change_by_3_actions  \
game_id action_id                                   
3857254 3                                   False   
        13                                  False   
        15                                  False   
        19                                  False   
        31                                  False   
...                                           ...   
3869685 2913                                False   
        2919                                False   
        2930                                False   
        2937                                False   
        2940                                False   

                   possession_change_by_4_actions  \
game_id action_id                                   
3857254 3                                   False   
        13                                  False   
        15                                  False   
        19                                  False   
        31                                  False   
...                                           ...   
3869685 2913                                False   
        2919                                False   
        2930                                False   
        2937                                False   
        2940                                False   

                   possession_change_by_5_actions  \
game_id action_id                                   
3857254 3                                   False   
        13                                  False   
        15                                  False   
        19                                  False   
        31                                  False   
...                          

In [12]:
for col in train_dataset.labels.columns:
    print(f"{col} : {train_dataset.labels[col].value_counts().to_dict()}")

concede_shots : {False: 30013, True: 3272}
possession_change_by_seconds : {False: 25934, True: 7351}
possession_change_by_1_actions : {False: 33285}
possession_change_by_2_actions : {False: 32495, True: 790}
possession_change_by_3_actions : {False: 27790, True: 5495}
possession_change_by_4_actions : {False: 24228, True: 9057}
possession_change_by_5_actions : {False: 21530, True: 11755}
possession_change_by_6_actions : {False: 19523, True: 13762}
possession_change_by_7_actions : {False: 17658, True: 15627}
possession_change_by_8_actions : {True: 17014, False: 16271}
possession_change_by_9_actions : {True: 18394, False: 14891}
possession_change_by_10_actions : {True: 19549, False: 13736}
possession_change_by_1_actions_and_5m_distance : {False: 33285}
possession_change_by_2_actions_and_5m_distance : {False: 33246, True: 39}
possession_change_by_3_actions_and_5m_distance : {False: 32510, True: 775}
possession_change_by_4_actions_and_5m_distance : {False: 32058, True: 1227}
possession_chang

In [17]:
for col in test_dataset.labels.columns:
    print(f"{col} : {test_dataset.labels[col].value_counts().to_dict()}")

concede_shots : {False: 11977, True: 1028}
possession_change_by_seconds : {False: 10056, True: 2949}
possession_change_by_1_actions : {False: 13005}
possession_change_by_2_actions : {False: 12704, True: 301}
possession_change_by_3_actions : {False: 10684, True: 2321}
possession_change_by_4_actions : {False: 9258, True: 3747}
possession_change_by_5_actions : {False: 8206, True: 4799}
possession_change_by_6_actions : {False: 7379, True: 5626}
possession_change_by_7_actions : {False: 6655, True: 6350}
possession_change_by_8_actions : {True: 6924, False: 6081}
possession_change_by_9_actions : {True: 7468, False: 5537}
possession_change_by_10_actions : {True: 7907, False: 5098}
possession_change_by_1_actions_and_5m_distance : {False: 13005}
possession_change_by_2_actions_and_5m_distance : {False: 12986, True: 19}
possession_change_by_3_actions_and_5m_distance : {False: 12621, True: 384}
possession_change_by_4_actions_and_5m_distance : {False: 12399, True: 606}
possession_change_by_5_actions